In [ ]:
#1 COM port chooser

import serial.tools.list_ports as list_ports

ports = list(list_ports.comports())

# Show ports with numbers
print("Available COM Ports:")
for i, p in enumerate(ports):
    print(f"{i}: {p.device} — {p.description}")

# User selects the port
selection = int(input("Select the COM port number: "))
selected_port = ports[selection].device

print(f"\n✅ Selected port: {selected_port}")


In [ ]:
#2 GRBL Connect call

import serial
import time

# Close existing connection if it exists
try:
    grbl.close()
except:
    pass

# (Re)connect to GRBL
baud = 115200
grbl = serial.Serial(selected_port, baud)
time.sleep(2)
grbl.flushInput()

print(f"🔗 Connected to {selected_port}")


In [ ]:
#3 GRBL settings + center position

grbl.write(b"$$\n")
time.sleep(1)

settings = {}
while grbl.in_waiting:
    line = grbl.readline().decode().strip()
    if line.startswith('$'):
        parts = line.split('=')
        if len(parts) == 2:
            key, val = parts
            settings[key] = float(val)

# Extract bed size from GRBL settings
bed_x = settings.get('$130', 300.0)  # default fallback
bed_y = settings.get('$131', 200.0)

center_x = bed_x / 2
center_y = bed_y / 2

print(f"🛏 Bed size: X={bed_x}mm, Y={bed_y}mm")
print(f"🎯 Center position: X={center_x}, Y={center_y}")


In [ ]:
grbl.write(b"$H\n")
time.sleep(5)

grbl.write(b"G91\n")
time.sleep(0.1)
grbl.write(b"G1 X5 Y5 F600\n")
time.sleep(1)

grbl.write(b"G90\n")
time.sleep(0.1)
grbl.write(f"G1 X{center_x} Y{center_y} F1500\n".encode())
time.sleep(2)

grbl.write(b"G91\n")
grbl.write(b"G1 X10 F500\n"); time.sleep(1)
grbl.write(b"G1 X-10 F500\n"); time.sleep(1)
grbl.write(b"G1 Y10 F500\n"); time.sleep(1)
grbl.write(b"G1 Y-10 F500\n"); time.sleep(1)

grbl.write(b"G90\n")
grbl.write(b"G1 X0 Y0 F1500\n"); time.sleep(3)

print("✅ Test complete.")


In [4]:
from IPython.display import display, Javascript
import ipywidgets as widgets
import time

# Reconnect if needed
try:
    grbl
except NameError:
    import serial
    selected_port = 'COM3'  # Set this to your port!
    grbl = serial.Serial(selected_port, 115200)
    time.sleep(2)
    grbl.flushInput()

center_x = 150
center_y = 100
jog_step = 5
jog_speed = 1000

status_output = widgets.Output()

def send_jog_command(axis, distance):
    try:
        grbl.write(b"G91\n")
        time.sleep(0.05)
        grbl.write(f"G1 {axis}{distance} F{jog_speed}\n".encode())
        time.sleep(0.05)
        grbl.write(b"G90\n")
        with status_output:
            print(f"Jog: {axis}{distance}")
    except Exception as e:
        with status_output:
            print(f"❌ Error: {e}")

# ✅ FIXED: These now match Y gantry motion convention
def jog_up(change=None): send_jog_command("Y", -jog_step)   # Move gantry up
def jog_down(change=None): send_jog_command("Y", jog_step)  # Move gantry down
def jog_left(change=None): send_jog_command("X", -jog_step)
def jog_right(change=None): send_jog_command("X", jog_step)
def go_home(change=None): grbl.write(b"$H\n"); time.sleep(5)
def go_center(change=None): grbl.write(f"G90\nG1 X{center_x} Y{center_y} F1500\n".encode())

# Buttons
btn_up = widgets.Button(description='↑')
btn_down = widgets.Button(description='↓')
btn_left = widgets.Button(description='←')
btn_right = widgets.Button(description='→')
btn_home = widgets.Button(description='🏠 Home')
btn_center = widgets.Button(description='🎯 Center')

btn_up.on_click(jog_up)
btn_down.on_click(jog_down)
btn_left.on_click(jog_left)
btn_right.on_click(jog_right)
btn_home.on_click(go_home)
btn_center.on_click(go_center)

# Keyboard activation
def enable_keyboard(change=None):
    display(Javascript('''
    (() => {
        if (window._grblKeyboardBound) return;
        window._grblKeyboardBound = true;
        document.addEventListener('keydown', (event) => {
            const kernel = IPython.notebook.kernel;
            const keyMap = {
                'ArrowUp': "jog_up()",
                'ArrowDown': "jog_down()",
                'ArrowLeft': "jog_left()",
                'ArrowRight': "jog_right()",
                '8': "jog_up()",
                '2': "jog_down()",
                '4': "jog_left()",
                '6': "jog_right()",
                'w': "jog_up()",
                's': "jog_down()",
                'a': "jog_left()",
                'd': "jog_right()"
            };
            const cmd = keyMap[event.key];
            if (cmd) {
                kernel.execute(cmd);
            }
        });
        alert("✅ Keyboard jogging enabled.\nClick here and use arrows, WASD, or 8462.");
    })();
    '''))

btn_enable_keys = widgets.Button(description='🎹 Enable Keyboard Jogging')
btn_enable_keys.on_click(enable_keyboard)

# Layout
jog_grid = widgets.GridBox(
    children=[widgets.Label(""), btn_up, widgets.Label(""),
              btn_left, widgets.Label(""), btn_right,
              widgets.Label(""), btn_down, widgets.Label("")],
    layout=widgets.Layout(
        grid_template_columns="100px 100px 100px",
        grid_template_rows="50px 50px 50px",
        justify_items="center"
    )
)
controls = widgets.HBox([btn_home, btn_center, btn_enable_keys])
gui = widgets.VBox([jog_grid, controls, status_output])
display(gui)


SerialException: could not open port 'COM3': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [ ]:
# Fire button using press/release detection via `observe`
btn_fire = widgets.ToggleButton(
    value=False,
    description='🔥 FIRE',
    button_style='danger',
    layout=widgets.Layout(width='100px', height='40px')
)

def on_fire_change(change):
    if change['new']:  # Pressed
        try:
            grbl.write(b"M3 S500\n")
            with status_output:
                print("🔥 Laser ON")
        except Exception as e:
            with status_output:
                print(f"❌ Laser start error: {e}")
    else:  # Released
        try:
            grbl.write(b"M5\n")
            with status_output:
                print("🛑 Laser OFF")
        except Exception as e:
            with status_output:
                print(f"❌ Laser stop error: {e}")

btn_fire.observe(on_fire_change, names='value')

# Add fire button to GUI
gui.children = list(gui.children) + [widgets.HBox([btn_fire])]
display(gui)


In [2]:
import ipywidgets as widgets


In [3]:
# Fire button using press/release detection via `observe`
btn_fire = widgets.ToggleButton(
    value=False,
    description='🔥 FIRE',
    button_style='danger',
    layout=widgets.Layout(width='100px', height='40px')
)

def on_fire_change(change):
    if change['new']:  # Pressed
        try:
            grbl.write(b"M3 S500\n")
            with status_output:
                print("🔥 Laser ON")
        except Exception as e:
            with status_output:
                print(f"❌ Laser start error: {e}")
    else:  # Released
        try:
            grbl.write(b"M5\n")
            with status_output:
                print("🛑 Laser OFF")
        except Exception as e:
            with status_output:
                print(f"❌ Laser stop error: {e}")

btn_fire.observe(on_fire_change, names='value')

# Add fire button to GUI
gui.children = list(gui.children) + [widgets.HBox([btn_fire])]
display(gui)



NameError: name 'gui' is not defined

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create fire button
btn_fire = widgets.ToggleButton(
    value=False,
    description='🔥 FIRE',
    button_style='danger',
    layout=widgets.Layout(width='100px', height='40px')
)

# Output box
status_output = widgets.Output()

# Laser control
def on_fire_change(change):
    if change['new']:
        try:
            grbl.write(b"M3 S500\n")
            with status_output:
                print("🔥 Laser ON")
        except Exception as e:
            with status_output:
                print(f"❌ Laser start error: {e}")
    else:
        try:
            grbl.write(b"M5\n")
            with status_output:
                print("🛑 Laser OFF")
        except Exception as e:
            with status_output:
                print(f"❌ Laser stop error: {e}")

btn_fire.observe(on_fire_change, names='value')

# Show it
display(widgets.VBox([btn_fire, status_output]))



In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Recreate status output if missing
try:
    status_output
except NameError:
    status_output = widgets.Output()

# Fire button
btn_fire = widgets.ToggleButton(
    value=False,
    description='🔥 FIRE',
    button_style='danger',
    layout=widgets.Layout(width='100px', height='40px')
)

def on_fire_change(change):
    if change['new']:  # Pressed
        try:
            grbl.write(b"M3 S500\n")
            with status_output:
                print("🔥 Laser ON")
        except Exception as e:
            with status_output:
                print(f"❌ Laser start error: {e}")
    else:  # Released
        try:
            grbl.write(b"M5\n")
            with status_output:
                print("🛑 Laser OFF")
        except Exception as e:
            with status_output:
                print(f"❌ Laser stop error: {e}")

btn_fire.observe(on_fire_change, names='value')

# Insert into existing GUI, or display alone
try:
    gui.children = list(gui.children) + [widgets.HBox([btn_fire])]
    display(gui)
except NameError:
    display(widgets.VBox([btn_fire, status_output]))


In [5]:
import ipywidgets as widgets
from IPython.display import display

# Ensure status output exists
try:
    status_output
except NameError:
    status_output = widgets.Output()

# Power slider: S0–S1000 (GRBL default max)
power_slider = widgets.IntSlider(
    value=500,
    min=0,
    max=1000,
    step=10,
    description='Power:',
    orientation='horizontal',
    layout=widgets.Layout(width='300px')
)

# FIRE button (momentary)
btn_fire = widgets.ToggleButton(
    value=False,
    description='🔥 FIRE',
    button_style='danger',
    layout=widgets.Layout(width='100px', height='40px')
)

# Laser control logic
def on_fire_change(change):
    power = power_slider.value
    if change['new']:  # Pressed
        try:
            grbl.write(f"M3 S{power}\n".encode())
            with status_output:
                print(f"🔥 Laser ON at S{power}")
        except Exception as e:
            with status_output:
                print(f"❌ Laser start error: {e}")
    else:  # Released
        try:
            grbl.write(b"M5\n")
            with stat


SyntaxError: expected ':' (3565174901.py, line 43)

In [8]:
import serial
import time

# Replace with your correct COM port
selected_port = 'COM5'  # <- Change if needed

# Establish serial connection
try:
    grbl = serial.Serial(selected_port, 115200)
    time.sleep(2)  # Allow GRBL to initialize
    grbl.flushInput()
    print("✅ GRBL connected on", selected_port)
except Exception as e:
    print("❌ Failed to connect to GRBL:", e)


✅ GRBL connected on COM5


In [10]:
import ipywidgets as widgets
from IPython.display import display

# Ensure status output exists
try:
    status_output
except NameError:
    status_output = widgets.Output()

# Power slider: S0–S1000 (GRBL default max)
power_slider = widgets.IntSlider(
    value=500,
    min=0,
    max=1000,
    step=10,
    description='Power:',
    orientation='horizontal',
    layout=widgets.Layout(width='300px')
)

# FIRE button (momentary)
btn_fire = widgets.ToggleButton(
    value=False,
    description='🔥 FIRE',
    button_style='danger',
    layout=widgets.Layout(width='100px', height='40px')
)

# Laser control logic
def on_fire_change(change):
    power = power_slider.value
    if change['new']:  # Pressed
        try:
            grbl.write(f"M3 S{power}\n".encode())
            with status_output:
                print(f"🔥 Laser ON at S{power}")
        except Exception as e:
            with status_output:
                print(f"❌ Laser start error: {e}")
    else:  # Released
        try:
            grbl.write(b"M5\n")
            with status_output:
                print("🛑 Laser OFF")
        except Exception as e:
            with status_output:
                print(f"❌ Laser stop error: {e}")

btn_fire.observe(on_fire_change, names='value')

# Combine slider + button
fire_controls = widgets.HBox([power_slider, btn_fire])

# Inject into GUI if available
try:
    gui.children = list(gui.children) + [fire_controls]
    display(gui)
except NameError:
    display(widgets.VBox([fire_controls, status_output]))
